Import dependance and load csv files

In [29]:
from cv2 import mean
import pandas as pd
import matplotlib.pyplot as plt
import datetime

stops = pd.read_csv('data_sncf/stops.txt')
stops_times = pd.read_csv('data_sncf/stop_times.txt')
routes = pd.read_csv('data_sncf/routes.txt')
trips = pd.read_csv('data_sncf/trips.txt')
calendar_dates = pd.read_csv('data_sncf/calendar_dates.txt')
calendar = pd.read_csv('data_sncf/calendar.txt')
agency = pd.read_csv('data_sncf/agency.txt')



Unify stop id format by keeping only the digits. This is necessary because the same station can have different character that are no interest for our purpose before the id. 

In [30]:
def formatID(id):
    return ''.join([i for i in id if i.isdigit()])


In [31]:
stops_times['stop_id']=stops_times['stop_id'].apply(formatID)


In [32]:
stops_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN037071R0100119847,23:05:00,23:05:00,87381509,0,NaN,0,0,NaN
1,OCESN037071R0100119847,23:35:00,23:35:00,87415604,1,NaN,0,0,NaN
2,OCESN037071R0100119847,23:55:00,23:55:00,87415620,2,NaN,0,0,NaN
3,OCESN037071R0100119847,24:25:00,24:25:00,87415877,3,NaN,0,0,NaN
4,OCESN037071R0100119847,24:45:00,24:45:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
209124,OCEZW861292F0200254327,18:15:00,18:15:00,87543165,13,NaN,0,0,NaN
209125,OCESN082201R04004~NAV55729,08:45:00,08:45:00,87471003,0,NaN,0,0,NaN
209126,OCESN082201R04004~NAV55729,09:55:00,09:55:00,87471607,1,NaN,0,0,NaN
209127,OCESN082201R03003~NAV55728,08:40:00,08:40:00,87471003,0,NaN,0,0,NaN


# Add date and day of availability for each trip



### Format service to load it neo4j :
calendar_dates contains the exceptions in the calendar where the service is unavailable. The meaning of exception type stays unknown.

In [33]:
calendar_dates

,service_id,date,exception_type
0,9939,20200413,2
1,8655,20200418,1
2,8655,20200516,1
3,12563,20200425,1
4,12563,20200508,1
...,...,...,...
38155,5931,20200501,2
38156,5931,20200508,2
38157,12834,20200413,2
38158,12834,20200501,2


In [34]:
#Groups all the exceptions by service id.
calendar_dates["date"]=calendar_dates["date"].astype(str)
calendar_dates=calendar_dates.groupby("service_id",as_index=False)["date"].sum()
calendar_dates['date']=calendar_dates['date'].map(lambda x:','.join([x[i:i+8]for i in range(0,len(x),8)]))
calendar_dates

,service_id,date
0,8,20200328
1,27,20200322
2,36,20200504
3,40,20200314
4,42,20200222
...,...,...
4275,12828,20200508
4276,12829,"20200224,20200225,20200226,20200227,20200228,2..."
4277,12832,20200413
4278,12833,"20200413,20200412"


**calendar** represents the period of availability of the service and the days on which it is available.

In [35]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,9939,1,1,1,1,0,0,0,20200220,20200520
1,8655,0,0,0,0,0,1,0,20200314,20200405
2,2341,0,0,0,0,0,1,0,20200222,20200307
3,12564,0,0,0,0,0,0,1,20200419,20200426
4,12565,0,0,0,0,0,1,0,20200314,20200404
...,...,...,...,...,...,...,...,...,...,...
5010,11448,1,1,1,1,1,1,0,20200222,20200520
5011,11283,1,1,1,1,1,0,0,20200224,20200520
5012,5931,1,1,1,1,1,0,0,20200220,20200520
5013,12834,1,1,1,1,1,0,0,20200220,20200520


Create a list of days from the dataframe.

In [36]:
calendar["days"]=calendar.apply(lambda x : ','.join([i for i in ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"] if x[i]==1]),axis=1)
calendar.loc[calendar['days']==','.join(["monday","tuesday","wednesday","thursday","friday","saturday","sunday"])]
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days
0,9939,1,1,1,1,0,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday"
1,8655,0,0,0,0,0,1,0,20200314,20200405,saturday
2,2341,0,0,0,0,0,1,0,20200222,20200307,saturday
3,12564,0,0,0,0,0,0,1,20200419,20200426,sunday
4,12565,0,0,0,0,0,1,0,20200314,20200404,saturday
...,...,...,...,...,...,...,...,...,...,...,...
5010,11448,1,1,1,1,1,1,0,20200222,20200520,"monday,tuesday,wednesday,thursday,friday,saturday"
5011,11283,1,1,1,1,1,0,0,20200224,20200520,"monday,tuesday,wednesday,thursday,friday"
5012,5931,1,1,1,1,1,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday,friday"
5013,12834,1,1,1,1,1,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday,friday"


Merge calendar and calendar_dates in order to have all the informations about the services in one dataframe. 

In [37]:
df_merged = calendar.merge(calendar_dates, on='service_id', how='outer')

# Concatenate the other columns when there are multiple rows with the same ID
#df_merged = df_merged.groupby('service_id').agg(lambda x: ','.join(x))

df_merged

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days,date
0,9939,1.0,1.0,1.0,1.0,0.0,0.0,0.0,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413
1,8655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200405.0,saturday,"20200418,20200516"
2,2341,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200222.0,20200307.0,saturday,NaN
3,12564,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20200419.0,20200426.0,sunday,NaN
4,12565,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200404.0,saturday,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5618,12817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200426
5619,12818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200413
5620,12823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200508
5621,12828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200508


Some service_id are in calendar but not in calendar_date and some service_id are in calendar_date but not in calendar.

So we will replace nan value in days by all the day of the week ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
The value in date will be replaced by an empy array

In [38]:
df_merged["days"]=df_merged["days"].fillna(",".join(["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]))

The value in date (which represents the unaivable dates) will be replaced by an empy array.

In [39]:
df_merged['date']=df_merged["date"].fillna("none")

The nan start dates will be replaced by 20200220 which is the first start date.
The nan end dates will be replaced by 20200520 which is the last end date.

In [40]:
df_merged['start_date']=df_merged['start_date'].fillna(20200220)
df_merged['end_date']=df_merged['end_date'].fillna(20200520)

In [41]:
df_merged

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days,date
0,9939,1.0,1.0,1.0,1.0,0.0,0.0,0.0,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413
1,8655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200405.0,saturday,"20200418,20200516"
2,2341,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200222.0,20200307.0,saturday,none
3,12564,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20200419.0,20200426.0,sunday,none
4,12565,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200404.0,saturday,none
...,...,...,...,...,...,...,...,...,...,...,...,...
5618,12817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200426
5619,12818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200413
5620,12823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200508
5621,12828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200508


# Create trips between station

In the stops_times dataframe there is the arrival time and the departure time of each trip in each stations.

In [74]:
stops_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN037071R0100119847,23:05:00,23:05:00,87381509,0,NaN,0,0,NaN
1,OCESN037071R0100119847,23:35:00,23:35:00,87415604,1,NaN,0,0,NaN
2,OCESN037071R0100119847,23:55:00,23:55:00,87415620,2,NaN,0,0,NaN
3,OCESN037071R0100119847,24:25:00,24:25:00,87415877,3,NaN,0,0,NaN
4,OCESN037071R0100119847,24:45:00,24:45:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
209124,OCEZW861292F0200254327,18:15:00,18:15:00,87543165,13,NaN,0,0,NaN
209125,OCESN082201R04004~NAV55729,08:45:00,08:45:00,87471003,0,NaN,0,0,NaN
209126,OCESN082201R04004~NAV55729,09:55:00,09:55:00,87471607,1,NaN,0,0,NaN
209127,OCESN082201R03003~NAV55728,08:40:00,08:40:00,87471003,0,NaN,0,0,NaN


Grouping information to make it easier to create a trip between each station.

In [71]:
rel=stops_times.groupby('trip_id',as_index=False)['arrival_time','departure_time','stop_id'].sum()

/tmp/ipykernel_6848/2245694226.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rel=stops_times.groupby('trip_id',as_index=False)['arrival_time','departure_time','stop_id'].sum()


In [72]:
rel

,trip_id,arrival_time,departure_time,stop_id
0,OCESN003100F140147152,05:20:0005:35:0005:49:0006:12:0007:38:00,05:20:0005:36:0005:50:0006:15:0007:38:00,8741301387413344874133858741101787384008
1,OCESN003100F150157151,05:20:0005:35:0005:49:0006:12:0007:40:00,05:20:0005:36:0005:50:0006:15:0007:40:00,8741301387413344874133858741101787384008
2,OCESN003101F050057154,06:40:0008:02:0008:25:0008:39:0008:54:00,06:40:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
3,OCESN003101F05005~NIB54832,06:40:0008:02:0008:25:0008:39:0008:54:00,06:40:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
4,OCESN003101F060067153,06:38:0008:02:0008:25:0008:39:0008:54:00,06:38:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
...,...,...,...,...
24106,OCEZW861293F0200254328,17:32:0017:42:0017:47:0017:55:0017:59:0018:04:...,17:32:0017:43:0017:48:0017:55:0017:59:0018:06:...,8754316587576082875761408757615787576165875761...
24107,OCEZW861295F0200254329,18:42:0018:52:0018:57:0019:05:0019:09:0019:14:...,18:42:0018:53:0018:58:0019:05:0019:09:0019:16:...,8754316587576082875761408757615787576165875761...
24108,OCEZW861296F0200254330,18:44:0018:51:0018:57:0019:04:0019:11:0019:13:...,18:44:0018:51:0018:58:0019:05:0019:11:0019:13:...,8757621587576231875762238757687687576199876992...
24109,OCEZW861297F0200254331,18:53:0019:03:0019:08:0019:16:0019:20:0019:25:...,18:53:0019:04:0019:09:0019:16:0019:20:0019:27:...,8754316587576082875761408757615787576165875761...


Separating the sums. (there is probably a better way but I couldn't find it)

In [75]:
rel['arrival_time']=rel['arrival_time'].map(lambda x:[x[i:i+8]for i in range(8,len(x),8)])
rel['departure_time']=rel['departure_time'].map(lambda x:[x[i:i+8]for i in range(0,len(x)-8,8)])
rel['stop_id']=rel['stop_id'].map(lambda x:[x[i:i+8]for i in range(0,len(x),8)])

Now we group stop id to make couples of value representing the start and the destination of each trip.

In [76]:
rel['stop_id']=rel['stop_id'].map(lambda x:[[x[i],x[i+1]]for i in range(len(x)-1)])

In [77]:
rel

,trip_id,arrival_time,departure_time,stop_id
0,OCESN003100F140147152,"[05:35:00, 05:49:00, 06:12:00, 07:38:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [..."
1,OCESN003100F150157151,"[05:35:00, 05:49:00, 06:12:00, 07:40:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [..."
2,OCESN003101F050057154,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
3,OCESN003101F05005~NIB54832,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
4,OCESN003101F060067153,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:38:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
...,...,...,...,...
24106,OCEZW861293F0200254328,"[17:42:00, 17:47:00, 17:55:00, 17:59:00, 18:04...","[17:32:00, 17:43:00, 17:48:00, 17:55:00, 17:59...","[[87543165, 87576082], [87576082, 87576140], [..."
24107,OCEZW861295F0200254329,"[18:52:00, 18:57:00, 19:05:00, 19:09:00, 19:14...","[18:42:00, 18:53:00, 18:58:00, 19:05:00, 19:09...","[[87543165, 87576082], [87576082, 87576140], [..."
24108,OCEZW861296F0200254330,"[18:51:00, 18:57:00, 19:04:00, 19:11:00, 19:13...","[18:44:00, 18:51:00, 18:58:00, 19:05:00, 19:11...","[[87576215, 87576231], [87576231, 87576223], [..."
24109,OCEZW861297F0200254331,"[19:03:00, 19:08:00, 19:16:00, 19:20:00, 19:25...","[18:53:00, 19:04:00, 19:09:00, 19:16:00, 19:20...","[[87543165, 87576082], [87576082, 87576140], [..."


We are going to flatten each list in each columns to have a row per trip so we need to make trip id a list with the same size as the lists in the other columns. 

In [78]:
rel['trip_id']=rel.apply(lambda x : [x['trip_id']for i in range(len(x['stop_id']))],axis=1)

Dividing the couples of stop id to have a start and a destination.

In [79]:
rel['start']=rel.apply(lambda x : [x['stop_id'][i][0]for i in range(len(x['stop_id']))],axis=1)
rel['dest']=rel.apply(lambda x : [x['stop_id'][i][1]for i in range(len(x['stop_id']))],axis=1)

In [80]:
rel

,trip_id,arrival_time,departure_time,stop_id,start,dest
0,"[OCESN003100F140147152, OCESN003100F140147152,...","[05:35:00, 05:49:00, 06:12:00, 07:38:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [...","[87413013, 87413344, 87413385, 87411017]","[87413344, 87413385, 87411017, 87384008]"
1,"[OCESN003100F150157151, OCESN003100F150157151,...","[05:35:00, 05:49:00, 06:12:00, 07:40:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [...","[87413013, 87413344, 87413385, 87411017]","[87413344, 87413385, 87411017, 87384008]"
2,"[OCESN003101F050057154, OCESN003101F050057154,...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
3,"[OCESN003101F05005~NIB54832, OCESN003101F05005...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
4,"[OCESN003101F060067153, OCESN003101F060067153,...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:38:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
...,...,...,...,...,...,...
24106,"[OCEZW861293F0200254328, OCEZW861293F020025432...","[17:42:00, 17:47:00, 17:55:00, 17:59:00, 18:04...","[17:32:00, 17:43:00, 17:48:00, 17:55:00, 17:59...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."
24107,"[OCEZW861295F0200254329, OCEZW861295F020025432...","[18:52:00, 18:57:00, 19:05:00, 19:09:00, 19:14...","[18:42:00, 18:53:00, 18:58:00, 19:05:00, 19:09...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."
24108,"[OCEZW861296F0200254330, OCEZW861296F020025433...","[18:51:00, 18:57:00, 19:04:00, 19:11:00, 19:13...","[18:44:00, 18:51:00, 18:58:00, 19:05:00, 19:11...","[[87576215, 87576231], [87576231, 87576223], [...","[87576215, 87576231, 87576223, 87576876, 87576...","[87576231, 87576223, 87576876, 87576199, 87699..."
24109,"[OCEZW861297F0200254331, OCEZW861297F020025433...","[19:03:00, 19:08:00, 19:16:00, 19:20:00, 19:25...","[18:53:00, 19:04:00, 19:09:00, 19:16:00, 19:20...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."


Finally we flatten the dataframe in a new dataframe to have a row for each trip.

In [83]:
df=pd.DataFrame({'start':rel['start'].sum(),'dest':rel['dest'].sum(),'trip_id':rel['trip_id'].sum(),'departure_time':rel['departure_time'].sum(),'arrival_time':rel['arrival_time'].sum()})

In [84]:
df

,start,dest,trip_id,departure_time,arrival_time
0,87413013,87413344,OCESN003100F140147152,05:20:00,05:35:00
1,87413344,87413385,OCESN003100F140147152,05:36:00,05:49:00
2,87413385,87411017,OCESN003100F140147152,05:50:00,06:12:00
3,87411017,87384008,OCESN003100F140147152,06:15:00,07:38:00
4,87413013,87413344,OCESN003100F150157151,05:20:00,05:35:00
...,...,...,...,...,...
185013,87576173,87576165,OCEZW861298F0200254332,19:37:00,19:42:00
185014,87576165,87576157,OCEZW861298F0200254332,19:42:00,19:46:00
185015,87576157,87576140,OCEZW861298F0200254332,19:46:00,19:53:00
185016,87576140,87576082,OCEZW861298F0200254332,19:54:00,19:58:00


# Add service_id in the rel dataframe :

We need to have the service id in this dataframe in order to add the availability informations.

In the trips dataframe there is the service_id corresponding to each trip.

In [81]:
trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN
1,OCE1506035,8655,OCESN003191F020027314,3191,1,NaN,NaN
2,OCE1506035,12563,OCESN003191F06006~NIR54906,3191,1,NaN,NaN
3,OCE1506035,2341,OCESN003191F010017311,3191,1,NaN,NaN
4,OCE1506035,12564,OCESN003117F14014~NIR54870,3117,1,NaN,NaN
...,...,...,...,...,...,...,...
24106,OCEZW-87543165-87576215,11256,OCEZW861288F0300354322,861288,0,NaN,NaN
24107,OCEZW-87543165-87576215,513,OCEZW861296F0200254330,861296,0,NaN,NaN
24108,OCEZW-87543165-87576215,540,OCEZW861292F0200254327,861292,0,NaN,NaN
24109,OCESN-87471607-87471003,12834,OCESN082201R04004~NAV55729,82201,1,NaN,NaN


We need to merge those dataframe in order to have the avaibility data of each trip.

In [85]:

stop_trip=trips.merge(df, on='trip_id')
stop_trip

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,start,dest,departure_time,arrival_time
0,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN,87381509,87415604,23:05:00,23:35:00
1,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN,87415604,87415620,23:35:00,23:55:00
2,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN,87415620,87415877,23:55:00,24:25:00
3,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN,87415877,87411207,24:25:00,24:45:00
4,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN,87411207,87411017,24:45:00,25:10:00
...,...,...,...,...,...,...,...,...,...,...,...
185013,OCEZW-87543165-87576215,540,OCEZW861292F0200254327,861292,0,NaN,NaN,87576157,87576140,17:52:00,17:59:00
185014,OCEZW-87543165-87576215,540,OCEZW861292F0200254327,861292,0,NaN,NaN,87576140,87576082,18:00:00,18:04:00
185015,OCEZW-87543165-87576215,540,OCEZW861292F0200254327,861292,0,NaN,NaN,87576082,87543165,18:05:00,18:15:00
185016,OCESN-87471607-87471003,12834,OCESN082201R04004~NAV55729,82201,1,NaN,NaN,87471003,87471607,08:45:00,09:55:00


In [87]:
stop_trip[['service_id','trip_id','start','dest','departure_time','arrival_time']]

,service_id,trip_id,start,dest,departure_time,arrival_time
0,9939,OCESN037071R0100119847,87381509,87415604,23:05:00,23:35:00
1,9939,OCESN037071R0100119847,87415604,87415620,23:35:00,23:55:00
2,9939,OCESN037071R0100119847,87415620,87415877,23:55:00,24:25:00
3,9939,OCESN037071R0100119847,87415877,87411207,24:25:00,24:45:00
4,9939,OCESN037071R0100119847,87411207,87411017,24:45:00,25:10:00
...,...,...,...,...,...,...
185013,540,OCEZW861292F0200254327,87576157,87576140,17:52:00,17:59:00
185014,540,OCEZW861292F0200254327,87576140,87576082,18:00:00,18:04:00
185015,540,OCEZW861292F0200254327,87576082,87543165,18:05:00,18:15:00
185016,12834,OCESN082201R04004~NAV55729,87471003,87471607,08:45:00,09:55:00


Now let's merge this dataframe with the df_merged dataframe where the information about the availability of each service is located.

In [89]:
final=df_merged[['service_id', 'start_date','end_date','days','date']].merge(stop_trip[['service_id','trip_id','start','dest','departure_time','arrival_time']], on='service_id')

In [96]:
final['arrival_time'].max()

'26:48:00'

In [94]:
df_info = pd.DataFrame(final.dtypes).T.rename(index= {0: 'column type'})
df_info = df_info.append(pd.DataFrame(final.isnull().sum()).T.rename(index= {0: 'null values (nb)'}))
df_info = df_info.append(pd.DataFrame(final.isnull().sum() / final.shape[0] * 100).T.rename(index= {0: "null values (%)"}))
df_info

/tmp/ipykernel_6848/3843462499.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_info = df_info.append(pd.DataFrame(final.isnull().sum()).T.rename(index= {0: 'null values (nb)'}))
/tmp/ipykernel_6848/3843462499.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_info = df_info.append(pd.DataFrame(final.isnull().sum() / final.shape[0] * 100).T.rename(index= {0: "null values (%)"}))


,service_id,start_date,end_date,days,date,trip_id,start,dest,departure_time,arrival_time
column type,int64,float64,float64,object,object,object,object,object,object,object
null values (nb),0,0,0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
final.to_csv('rel.csv')

### Load data in neo4j

Put rel.csv in the import directory of your dbms. (on ubuntu "~/.config/'Neo4j Desktop'/Application/relate-data/dbmss/dbms-<dbmss id>/import")

Open neo4j browser then type
```
LOAD CSV WITH HEADERS FROM "file:///rel.csv" AS row with split(row.departure_time,":") as dep,split(row.arrival_time,":") as ar, row
MATCH (s1:Station {id: row.start})
MATCH (s2:Station {id: row.dest})
MERGE (s1)-[r:TRIP_TO {departure_time:row.departure_time,arrival_time:row.arrival_time, start_date:row.start_date, end_date:row.end_date, available_days:row.days, unavailability:row.date, trip_id:row.trip_id, duration:(toFloat(ar[0])-toFloat(dep[0]))*60+toFloat(ar[1])-toFloat(dep[1])}]-(s2)
```


# Implement custom astar

## add latitude and longitute to final dataframe

In [102]:
stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,87381509,Gare de Mantes-la-Jolie,NaN,48.989687,1.703294,NaN,NaN,1,NaN
1,87415604,Gare de Vernon-Giverny,NaN,49.091286,1.478363,NaN,NaN,1,NaN
2,87415620,Gare de Gaillon-Aubevoye,NaN,49.174632,1.352518,NaN,NaN,1,NaN
3,87415877,Gare de Val-de-Reuil,NaN,49.275399,1.224609,NaN,NaN,1,NaN
4,87411207,Gare de Oissel,NaN,49.343042,1.101821,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
3831,87718403,Gare de Roche-lez-Beaupré,NaN,47.276417,6.111272,NaN,NaN,1,NaN
3832,87718411,Gare de Novillars,NaN,47.283906,6.132791,NaN,NaN,1,NaN
3833,87718429,Gare de Deluz,NaN,47.292454,6.199545,NaN,NaN,1,NaN
3834,87718437,Gare de Laissey,NaN,47.298881,6.233690,NaN,NaN,1,NaN


In [98]:
stops['stop_id']=stops['stop_id'].apply(formatID)
stops=stops.drop_duplicates(subset=['stop_id'])
stops=stops.dropna(subset=['stop_name'])

In [104]:
start=stops[['stop_id','stop_lat','stop_lon']].rename(columns={'stop_id':'start','stop_lon':'start_lon','stop_lat':'start_lat'})

In [105]:
start

,start,start_lat,start_lon
0,87381509,48.989687,1.703294
1,87415604,49.091286,1.478363
2,87415620,49.174632,1.352518
3,87415877,49.275399,1.224609
4,87411207,49.343042,1.101821
...,...,...,...
3831,87718403,47.276417,6.111272
3832,87718411,47.283906,6.132791
3833,87718429,47.292454,6.199545
3834,87718437,47.298881,6.233690


In [107]:
final=final.merge(start,on='start')

In [110]:
final

,service_id,start_date,end_date,days,date,trip_id,start,dest,departure_time,arrival_time,start_lat,start_lon
0,9939,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413,OCESN037071R0100119847,87381509,87415604,23:05:00,23:35:00,48.989687,1.703294
1,2341,20200222.0,20200307.0,saturday,none,OCESN013103F2202215425,87381509,87415604,06:51:00,07:03:00,48.989687,1.703294
2,2341,20200222.0,20200307.0,saturday,none,OCESN013127F1101115517,87381509,87415604,17:51:00,18:03:00,48.989687,1.703294
3,2341,20200222.0,20200307.0,saturday,none,OCESN013104F0600615429,87381509,87384008,07:10:00,07:50:00,48.989687,1.703294
4,2341,20200222.0,20200307.0,saturday,none,OCESN013132F0800815538,87381509,87384008,19:10:00,19:50:00,48.989687,1.703294
...,...,...,...,...,...,...,...,...,...,...,...,...
185007,1827,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...","20200418,20200419,20200224,20200225,20200226,2...",OCESN890212F0200251314,87721704,87721480,09:38:00,09:40:00,45.777326,4.753923
185008,1827,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...","20200418,20200419,20200224,20200225,20200226,2...",OCESN890208F0200251312,87721704,87721480,08:36:00,08:38:00,45.777326,4.753923
185009,228,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200223,OCESN849909F3203241791,87271585,87271585,09:56:00,09:57:00,49.200893,2.838112
185010,228,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200223,OCESN849909F3203241791,87271585,87271585,10:07:00,10:08:00,49.200893,2.838112


In [108]:
dest=stops[['stop_id','stop_lat','stop_lon']].rename(columns={'stop_id':'dest','stop_lon':'dest_lon','stop_lat':'dest_lat'})

In [109]:
dest

,dest,dest_lat,dest_lon
0,87381509,48.989687,1.703294
1,87415604,49.091286,1.478363
2,87415620,49.174632,1.352518
3,87415877,49.275399,1.224609
4,87411207,49.343042,1.101821
...,...,...,...
3831,87718403,47.276417,6.111272
3832,87718411,47.283906,6.132791
3833,87718429,47.292454,6.199545
3834,87718437,47.298881,6.233690


In [111]:
final=final.merge(dest,on='dest')

In [112]:
final

,service_id,start_date,end_date,days,date,trip_id,start,dest,departure_time,arrival_time,start_lat,start_lon,dest_lat,dest_lon
0,9939,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413,OCESN037071R0100119847,87381509,87415604,23:05:00,23:35:00,48.989687,1.703294,49.091286,1.478363
1,2341,20200222.0,20200307.0,saturday,none,OCESN013103F2202215425,87381509,87415604,06:51:00,07:03:00,48.989687,1.703294,49.091286,1.478363
2,2341,20200222.0,20200307.0,saturday,none,OCESN013127F1101115517,87381509,87415604,17:51:00,18:03:00,48.989687,1.703294,49.091286,1.478363
3,4047,20200222.0,20200307.0,"saturday,sunday",none,OCESN013131F1001015532,87381509,87415604,18:51:00,19:03:00,48.989687,1.703294,49.091286,1.478363
4,5545,20200314.0,20200405.0,saturday,"20200418,20200516",OCESN013127F0700715519,87381509,87415604,17:51:00,18:03:00,48.989687,1.703294,49.091286,1.478363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185004,1968,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday","20200413,20200501,20200508",OCESN859073F0200244788,87486324,87486316,07:57:00,08:01:00,46.651484,-1.178474,46.665367,-1.231964
185005,1968,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday","20200413,20200501,20200508",OCESN859071F0200244786,87486324,87486316,06:41:00,06:45:00,46.651484,-1.178474,46.665367,-1.231964
185006,9174,20200223.0,20200426.0,sunday,20200413,OCESN859077F0200244792,87486324,87486316,19:31:00,19:35:00,46.651484,-1.178474,46.665367,-1.231964
185007,2194,20200222.0,20200516.0,saturday,"20200321,20200328,20200404,20200411",OCESN849953F0200241918,87271874,87271882,06:58:00,07:02:00,49.403745,3.357268,49.439473,3.402945
